

- Transfer Learning with TF 2.0 (https://towardsdatascience.com/transfer-learning-with-tf-2-0-ff960901046d)
- Models pre-trained on ImageNet (https://keras.io/applications/)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, metrics

import numpy as np
import shutil
from random import shuffle
import os
import pathlib
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls "MODEL2/train/LEVEL1" | wc -l
!ls "MODEL2/train/LEVEL2" | wc -l
!ls "MODEL2/train/LEVEL3" | wc -l

!ls "MODEL2/test/LEVEL1" | wc -l
!ls "MODEL2/test/LEVEL2" | wc -l
!ls "MODEL2/test/LEVEL3" | wc -l

2992
2992
2992
748
748
748


In [ ]:
!ls  'MODEL2_2/images/-7.937391576878188_-7.9351502650639025_37.02194325529446_37.023745413806296.jpg'

MODEL2_2/images/-7.937391576878188_-7.9351502650639025_37.02194325529446_37.023745413806296.jpg


In [ ]:
!cp  'drive/My Drive/datasets/MODEL2_2/model2_2.tar.gz' 'model2_2.tar.gz'
!tar -xvf  'model2_2.tar.gz' 

Streaming output truncated to the last 5000 lines.
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.64726620245071_38.64906786249465.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.64906786249465_38.65086952198221.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.65086952198221_38.65267118091338.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.704919047853046_38.70672069008896.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.70672069008896_38.70852233176823.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.71032397289088_38.712125613456884.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.713927253466224_38.71572889291893.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.71933217015432_38.721133807936994.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.721133807936994_38.72293544516298.jpg
MODEL2_2/images/-9.147699956592684_-9.145458644778401_38.72293544516298_38.72473708183228.jpg
MODE

### Using image sizes of 224x224 with 3 channels (This is the default of the pre-trained model we are going to use, and so it's a good idea to keep the same sizes, because those where used to train them most-likely but other sizes can be used without much problem)

### Also the 80% / 20% split was used for trainning and validation

In [ ]:
NCLASSES = 1
HEIGHT = 128
WIDTH = 128
NUM_CHANNELS = 3
BATCH_SIZE = 32
TRAIN_PERC=1
TEST_PERC=1
PATH="MODEL2_2/images"
TRAIN_DATAFRAME = pd.read_csv("MODEL2_2/model2_2_train.csv")
TEST_DATAFRAME= pd.read_csv("MODEL2_2/model2_2_test.csv")
TRAIN_SET_COUNT = len(TRAIN_DATAFRAME) * TRAIN_PERC
TEST_SET_COUNT = len(TEST_DATAFRAME) * TEST_PERC
METRICS = [metrics.mean_absolute_error]
print((TRAIN_SET_COUNT,TEST_SET_COUNT))

(30335, 7583)


Load a model from keras database (https://keras.io/applications/) with weights trained on ImageNet

Note that the *image_top* parameter is set to *true*. This implies that we are only keeping the convolutional layers of the network

### The MobileNetV2 was chosen, it was developed by Google and it's an architecture that was suposed to bring neural networks to mobile, and because of that it should be not only good but fast and require less resoucers than others .

In [ ]:
#base_model = tf.keras.applications.vgg19.VGG19(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')
##base_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False,weights=None, input_shape=(HEIGHT, WIDTH, NUM_CHANNELS))
base_model = tf.keras.applications.ResNet50(include_top=False,weights="imagenet", input_shape=(HEIGHT, WIDTH, NUM_CHANNELS))
#base_model = tf.keras.applications.VGG16(include_top=False,weights=None, input_shape=(HEIGHT, WIDTH, NUM_CHANNELS))
#base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,weights=None, input_shape=(HEIGHT, WIDTH, NUM_CHANNELS))
##base_model = tf.keras.applications.InceptionV3(include_top=False,weights="imagenet", input_shape=(HEIGHT, WIDTH, NUM_CHANNELS))
MODEL_FILE="inception_dataset2_2.hdf5"
MODEL_HISTORY=f"drive/My Drive/history/{MODEL_FILE}"
MODEL_FILE=f"drive/My Drive/models/{MODEL_FILE}"
print(base_model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Freeze the preloaded weights, these won't be used for training
comment the line below to train the full network

In [ ]:
#base_model.trainable = False

Add new layers to the top of the network

Global Average Pooling (GAP) performs an average for each of the feature maps. Assuming the last conv layer is 3x3x256, we have 256 feature maps. Global average pooling is a layer with 256 values, each being the average of a feature map.

Original paper for Global Average Pooling: Network In Network, Min Lin1, Qiang Chen, Shuicheng Yan (https://arxiv.org/pdf/1312.4400.pdf). Note the GAP is not the main theme of the paper.

In [ ]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(1, activation='linear')(x)

model_3 = models.Model(inputs=base_model.input, outputs=x)
print(model_3.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
train_label_df = pd.read_csv("MODEL2_2/model2_2_train.csv")
train_label_df.dtypes

Unnamed: 0        int64
Unnamed: 0.1      int64
location         object
label           float64
dtype: object

Auxiliary function for loading the dataset

#### Changed to deal with JPEG and resize the images, because they have varying sizes and we want them to be all 224x224 with 3 channels

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_label_df = pd.read_csv("MODEL2_2/model2_2_train.csv")
test_label_df = pd.read_csv("MODEL2_2/model2_2_test.csv")

def transform_locations(df):
  locations = list()
  locations.append("location")
  for x in df['location'][1:]:
    _,image = x.split("/")
    locations.append(f"{image}")

  return locations

def transform_label(df):
  new_col = list()
  
  for x in df['label'][1:]:
    x = float(x)
    new_col.append(x)
  
  return new_col
 

train_label_df['location'] = transform_locations(train_label_df)
test_label_df['location'] = transform_locations(test_label_df)

train_label_df['label'] = train_label_df['label'] * 100
test_label_df['label'] = test_label_df['label'] * 100

print(pd.__version__)


##train_label_df['locations'] = transform_locations(train_label_df)
##test_label_df['locations'] = transform_locations(test_label_df)




train_dataset = train_datagen.flow_from_dataframe(dataframe=train_label_df, directory=PATH, 
                                              x_col="location", y_col="label",target_size=(WIDTH, HEIGHT), 
                                              batch_size=BATCH_SIZE,class_mode="raw")


test_dataset = train_datagen.flow_from_dataframe(dataframe=test_label_df, directory=PATH, 
                                              x_col="location", y_col="label",target_size=(WIDTH, HEIGHT), 
                                              batch_size=BATCH_SIZE,class_mode="raw")
X_train,y_train = train_dataset.next()
y_train.dtype

1.0.5
Found 30334 validated image filenames.
Found 7582 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="location". These filename(s) will be ignored.
  .format(n_invalid, x_col)


dtype('float64')

Split the dataset into training and validation sets

Load the test set

In [ ]:
def get_past_history(MODEL_HISTORY):
  
  if not os.path.isfile(MODEL_HISTORY):
    return None 
  else:
    history = pd.read_csv(MODEL_HISTORY)
    return history

def get_total_history(past_history,new_history):
  
  history = pd.concat([past_history,new_history], axis=0, join='outer', ignore_index=True, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)
  return history

def get_past_model(MODEL_FILE):
  
  if not os.path.isfile(MODEL_FILE):
    
    return None 
  else:
    model = load_model(MODEL_FILE)
    return model


In [ ]:
EPOCHS=2

past_history = get_past_history(MODEL_HISTORY)

checkpoint = ModelCheckpoint(MODEL_FILE, monitor='loss', verbose=1,
    save_best_only=False, mode='auto', save_freq='epoch')

In [ ]:
past_model = get_past_model(MODEL_FILE)
model_3 = past_model
history = past_history
'''
if past_model:
  model_3 = past_model
else:
  model_3.compile(optimizer='rmsprop', loss='mean_squared_error')

history = model_3.fit(train_dataset, epochs=EPOCHS, steps_per_epoch=TRAIN_SET_COUNT/BATCH_SIZE, validation_data = test_dataset, validation_steps= TEST_SET_COUNT/BATCH_SIZE,
                      callbacks=[checkpoint])


history = get_total_history(past_history,pd.DataFrame.from_dict(history.history))

history.to_csv(MODEL_HISTORY)
'''

"\nif past_model:\n  model_3 = past_model\nelse:\n  model_3.compile(optimizer='rmsprop', loss='mean_squared_error')\n\nhistory = model_3.fit(train_dataset, epochs=EPOCHS, steps_per_epoch=TRAIN_SET_COUNT/BATCH_SIZE, validation_data = test_dataset, validation_steps= TEST_SET_COUNT/BATCH_SIZE,\n                      callbacks=[checkpoint])\n\n\nhistory = get_total_history(past_history,pd.DataFrame.from_dict(history.history))\n\nhistory.to_csv(MODEL_HISTORY)\n"

Show how the learning progressed

##### It does look like that the model could still improve after 40 epochs by looking at the graph.

In [ ]:
##print(history.history.keys())

# summarize history for accuracy
plt.plot(history['mean_absolute_error'])
plt.plot(history['val_mean_absolute_error'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

KeyError: ignored

In [ ]:
model_3.evaluate(test_dataset)

71/71 [==============================] - 2s 29ms/step - loss: 1.0858 - categorical_accuracy: 0.3917


[1.0858261585235596, 0.3917112350463867]